In [1]:
# set environment and import packages
import os
from pandas import read_csv
import pandas as pd
import geopandas as gps
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import constants as c
import warnings
warnings.filterwarnings("ignore")

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync')



In [2]:
sctg_group_lookup = read_csv(c.input_dir + c.sctg_group_lookup_file, sep = ',')
mesozone_lookup = read_csv(c.input_dir + c.mesozone_id_lookup_file, sep = ',')

truck_mode = ['For-hire Truck', 'Private Truck']
# max_ton_lookup = {'sctg1': 218.327, 'sctg2': 51.13091, 'sctg3': 94.56035, 'sctg4': 4.4861, 'sctg5': 51.64694}

In [3]:
combined_modeled_OD = None
mode_choide_by_commodity = None
combined_truck_output = None
for sctg in c.list_of_sctg_group:
    print(sctg)
    b2b_dir = c.output_dir + sctg
    list_of_b2b_files = listdir(b2b_dir)
    iterator = 0
#     cut_off_point = max_ton_lookup[sctg] # shipment capacity for this shipment
    for file in list_of_b2b_files:
        if file == '.DS_Store':
            continue
        if iterator%10 == 0:
            print(iterator)
        modeled_OD_by_sctg = read_csv(b2b_dir + '/' + file, sep = ',')
#         print(modeled_OD_by_sctg.columns)
        list_of_var = ['BuyerID', 'BuyerZone', 'BuyerNAICS', 'SellerID',
           'SellerZone', 'SellerNAICS', 'TruckLoad', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID', 
           'mode_choice', 'probability', 'Distance', 'Travel_time']
#         print(modeled_OD_by_sctg.head(5))
        truck_output = modeled_OD_by_sctg.loc[modeled_OD_by_sctg['mode_choice'].isin(truck_mode), list_of_var]
        int_var = ['BuyerID', 'BuyerZone', 'SellerID',
           'SellerZone', 'SCTG_Group', 
           'shipment_id', 'orig_FAFID', 'dest_FAFID']
        truck_output.loc[:, int_var] = truck_output.loc[:, int_var].astype(int)
        truck_output.to_csv(c.output_dir + sctg + '_truck/truck_only_OD_' + sctg + '_' + 
                            str(iterator) + '.csv', index = False)

        
        ## compute national shipment count and tonmile
        modeled_OD_by_sctg['ShipmentLoad'] = modeled_OD_by_sctg['TruckLoad'] / 1000 # convert to thousand tons
        
        #print(cut_off_point)
#         modeled_OD_by_sctg.loc[modeled_OD_by_sctg['ShipmentLoad'] > cut_off_point, 'ShipmentLoad'] = cut_off_point
        modeled_OD_by_sctg['tmiles'] = modeled_OD_by_sctg['ShipmentLoad'] * 1000 * modeled_OD_by_sctg['Distance']
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['SellerZone', 'orig_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'left')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "orig_GEOID",
                                                                "CBPZONE": "orig_CBPZONE", 
                                                                "MESOZONE":"orig_MESOZONE", 
                                                                "FAFNAME":"orig_FAFNAME"})
        modeled_OD_by_sctg = pd.merge(modeled_OD_by_sctg, mesozone_lookup, 
                                      left_on = ['BuyerZone', 'dest_FAFID'], 
                                    right_on = ['MESOZONE', 'FAFID'], how = 'left')
        modeled_OD_by_sctg = modeled_OD_by_sctg.rename(columns={"GEOID": "dest_GEOID", 
                                                                "CBPZONE": "dest_CBPZONE", 
                                                                "MESOZONE":"dest_MESOZONE", 
                                                               "FAFNAME":"dest_FAFNAME"})    
        agg_OD_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['tmiles', 'ShipmentLoad']].sum()        
        agg_OD_by_sctg = agg_OD_by_sctg.reset_index()
        agg_count_by_sctg = modeled_OD_by_sctg.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'])[['shipment_id']].count() 
        agg_count_by_sctg = agg_count_by_sctg.reset_index()
        agg_OD_by_sctg = pd.merge(agg_OD_by_sctg, agg_count_by_sctg, 
                                  on = ["orig_FAFID", "orig_FAFNAME", "dest_FAFID", "dest_FAFNAME", "SCTG_Group", 'mode_choice'],
                                  how = 'left')
        agg_OD_by_sctg = agg_OD_by_sctg.rename(columns={"shipment_id": "count"})
        agg_OD_by_sctg.loc[:, 'SCTG_Name'] = c.sctg_def[sctg]
        agg_OD_by_sctg.loc[:, 'chunk_id'] = iterator
        combined_modeled_OD = pd.concat([combined_modeled_OD, agg_OD_by_sctg], sort = False)
        iterator += 1 
#         break        
#     break
#     combined_truck_output.to_csv(c.input_dir + 'truck_only_OD_' + sctg + '.csv', index = False)
# combined_modeled_OD = pd.merge(combined_modeled_OD, sctg_group_definition, on = ['SCTG_Group'], how = 'left')
combined_modeled_OD.head(10)    

sctg1
0
10
sctg2
0
10
sctg3
0
sctg4
0
10
20
30
40
50
60
sctg5
0


,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,mode_choice,tmiles,ShipmentLoad,count,SCTG_Name,chunk_id
0,190,Iowa,489,Rest of TX,1,For-hire Truck,7.908476e+05,0.799458,7,bulk,0
1,190,Iowa,489,Rest of TX,1,Rail/IMX,4.523133e+09,4223.951855,36983,bulk,0
2,209,Rest of KS,489,Rest of TX,1,For-hire Truck,1.012853e+08,137.285806,1202,bulk,0
3,209,Rest of KS,489,Rest of TX,1,Rail/IMX,1.131458e+10,10255.091151,89788,bulk,0
4,229,Rest of LA,489,Rest of TX,1,For-hire Truck,1.038311e+07,10.018637,342,bulk,0
5,229,Rest of LA,489,Rest of TX,1,Rail/IMX,5.070008e+06,4.073790,139,bulk,0
6,280,Mississippi,481,Austin,1,For-hire Truck,1.580624e+08,216.651507,7376,bulk,0
7,280,Mississippi,481,Austin,1,Rail/IMX,3.172115e+07,40.504687,1379,bulk,0
8,299,Rest of MO,489,Rest of TX,1,For-hire Truck,8.378570e+04,0.106404,2,bulk,0
9,299,Rest of MO,489,Rest of TX,1,Rail/IMX,2.204057e+05,0.215650,2,bulk,0


In [4]:
combined_modeled_OD_agg = combined_modeled_OD.groupby(["orig_FAFID", "orig_FAFNAME", "dest_FAFID", \
                                                       "dest_FAFNAME", "SCTG_Group", 'SCTG_Name',
                                                       'mode_choice'])[['tmiles', 'ShipmentLoad', 'count']].sum()
combined_modeled_OD_agg = combined_modeled_OD_agg.reset_index()
combined_modeled_OD_agg.head(5)
# combined_modeled_OD_agg.loc[:, 'in_study_area'] = 0
# buffer = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.bay_area_region_code) | \
#         combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.bay_area_region_code)
# combined_modeled_OD_agg.loc[buffer, 'in_study_area'] = 1

combined_modeled_OD_agg.loc[:, 'outbound'] = 0
combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'orig_FAFID'].isin(c.region_code), 'outbound'] = 1

combined_modeled_OD_agg.loc[:, 'inbound'] = 0
combined_modeled_OD_agg.loc[combined_modeled_OD_agg.loc[:, 'dest_FAFID'].isin(c.region_code), 'inbound'] = 1

combined_modeled_OD_agg.loc[:, 'orig_FAFID'] = combined_modeled_OD_agg.loc[:, 'orig_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'dest_FAFID'] = combined_modeled_OD_agg.loc[:, 'dest_FAFID'].astype(int)
combined_modeled_OD_agg.loc[:, 'SCTG_Group'] = combined_modeled_OD_agg.loc[:, 'SCTG_Group'].astype(int)
combined_modeled_OD_agg.loc[:, 'Distance'] = combined_modeled_OD_agg.loc[:, 'tmiles'] / 1000 / combined_modeled_OD_agg.loc[:, 'ShipmentLoad']
combined_modeled_OD_agg.head(10)

,orig_FAFID,orig_FAFNAME,dest_FAFID,dest_FAFNAME,SCTG_Group,SCTG_Name,mode_choice,tmiles,ShipmentLoad,count,outbound,inbound,Distance
0,11,Birmingham,481,Austin,4,mfr_goods,For-hire Truck,6.148288e+06,7.167842,1173,0,1,857.760000
1,11,Birmingham,481,Austin,4,mfr_goods,Rail/IMX,7.914276e+04,0.079545,13,0,1,994.940199
2,11,Birmingham,488,San Antonio,2,fuel_fert,For-hire Truck,1.081723e+06,1.142287,54,0,1,946.980000
3,11,Birmingham,488,San Antonio,2,fuel_fert,Rail/IMX,1.934711e+05,0.212202,10,0,1,911.730199
4,11,Birmingham,488,San Antonio,4,mfr_goods,For-hire Truck,3.152662e+08,332.917457,12204,0,1,946.980000
5,11,Birmingham,488,San Antonio,4,mfr_goods,Rail/IMX,2.138789e+08,234.585769,8376,0,1,911.730199
6,11,Birmingham,489,Rest of TX,4,mfr_goods,For-hire Truck,1.563199e+08,176.242314,13988,0,1,886.960000
7,11,Birmingham,489,Rest of TX,4,mfr_goods,Rail/IMX,1.766442e+07,19.221020,993,0,1,919.015760
8,12,Mobile,481,Austin,4,mfr_goods,Air,3.123159e+03,0.005399,5,0,1,578.423145
9,12,Mobile,481,Austin,4,mfr_goods,For-hire Truck,3.933574e+08,428.965868,76838,0,1,916.990000


In [5]:
combined_modeled_OD_agg.to_csv(c.output_dir + 'processed_b2b_flow_summary.csv', sep = ',')